# 한빛출판네트워크 - 새로나온책 10페이지
        - 제목, 저자, 출간일, 쪽수, 가격, 역자

In [8]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

### 1.데이터 가져오기

In [9]:
site = 'https://www.hanbit.co.kr'
url = site + '/store/books/new_book_list.html'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

### 2.원하는 데이터 찾기

In [10]:
#메인 페이지
lis = soup.select('.new_book_list_wrap > div > li')
len(lis)

20

In [11]:
li = lis[0]


In [16]:
#서브 페이지
href = li.select_one('.book_tit').find('a')['href']
url2 = site + href
res = requests.get(url2)
sub_soup = BeautifulSoup(res.text, 'html.parser')

In [25]:
sub_lis = sub_soup.select('.info_list > li')
sub_lis

[<li><strong>저자 : </strong><span>펠릭스 춤슈타인 </span></li>,
 <li><strong>번역 : </strong><span>한선용 </span></li>,
 <li><strong>출간 : </strong><span>2022-04-20</span></li>,
 <li><strong>페이지 : </strong><span>388 쪽</span></li>,
 <li><strong>ISBN : </strong><span>9791162245545</span></li>,
 <li><strong>물류코드 :</strong><span>10554</span></li>]

### 3.하나의 데이터로 원하는 정보 찾기

In [12]:
#제목
book_name = li.select_one('.book_tit').get_text()

In [31]:
#가격
price = sub_soup.select_one('.pbr').get_text()

'28,000원'

In [ ]:
#저자, 번역, 출간(날짜),페이지는 sub_soup 생성 후 불러오기
href = li.select_one('.book_tit').find('a')['href']
url2 = site + href
res = requests.get(url2)
sub_soup = BeautifulSoup(res.text, 'html.parser')
sub_lis = sub_soup.select('.info_list > li')

In [43]:
for li in sub_lis:
    print(li.get_text())

저자 : 펠릭스 춤슈타인 
번역 : 한선용 
출간 : 2022-04-20
페이지 : 388 쪽
ISBN : 9791162245545
물류코드 :10554


In [ ]:
text = li.get_text().strip()

if '저자 :' in text:

    author = text.split(':')[1].strip()

elif '번역 :' in text:
    trans = text.split(':')[1].strip()
elif '출간 :' in text:
    date = text.split(':')[1].strip()
elif '페이지 :' in text:
    page = text.split(':')[1].strip()

### 4.페이지내 모든 책 정보 가져오기

In [56]:
lines = []
for li in tqdm(lis):
    book_name = li.select_one('.book_tit').get_text()

    href = li.select_one('.book_tit').find('a')['href']
    url2 = site + href
    res = requests.get(url2)
    sub_soup = BeautifulSoup(res.text, 'html.parser')
    sub_lis = sub_soup.select('.info_list > li')
    
    price = sub_soup.select_one('.pbr').get_text()
    
    trans = '-'       
    for li in sub_lis:
        text = li.get_text().strip()
        if '저자 :' in text:
            author = text.split(':')[1].strip()
        elif '번역 :' in text:
            trans = text.split(':')[1].strip()
        elif '출간 :' in text:
            date = text.split(':')[1].strip()
        elif '페이지 :' in text:
            page = text.split(':')[1].strip()
        
    lines.append([book_name, author, date, page, price, trans])
        
    

  0%|          | 0/20 [00:00<?, ?it/s]

### 5.모든 페이지의 모든 정보 가져오기

In [64]:
lines2 = []
for page in tqdm(range(1,11)):
    url = f'https://www.hanbit.co.kr/store/books/new_book_list.html?page={page}&brand=&cate1=&cate2=&searchKey=&keyWord='
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select('li.sub_book_list')

    for li in lis:
        book_name = li.select_one('.book_tit').get_text()

        href = li.select_one('.book_tit').find('a')['href']
        url2 = site + href
        res = requests.get(url2)
        sub_soup = BeautifulSoup(res.text, 'html.parser')
        sub_lis = sub_soup.select('.info_list > li')
        
        price = sub_soup.select_one('.pbr').get_text()
        
        trans = '-'       
        for li in sub_lis:
            text = li.get_text().strip()
            if '저자 :' in text:
                author = text.split(':')[1].strip()
            elif '번역 :' in text:
                trans = text.split(':')[1].strip()
            elif '출간 :' in text:
                date = text.split(':')[1].strip()
            elif '페이지 :' in text:
                page = text.split(':')[1].strip()
            
        lines2.append([book_name, author, date, page, price, trans])

  0%|          | 0/10 [00:00<?, ?it/s]

In [65]:
df = pd.DataFrame(lines2, columns=['제목','저자','출간일','쪽수','가격','역자'])
df

,제목,저자,출간일,쪽수,가격,역자
0,엑셀이 편해지는 파이썬,펠릭스 춤슈타인,2022-04-20,388 쪽,"28,000원",한선용
1,올림포스 연대기,김재훈,2022-04-20,280 쪽,"18,000원",-
2,쉽게 배우는 AWS AI 서비스,"피터 엘거 , 오언 셔너히",2022-04-15,396 쪽,"30,000원","맹윤호 , 임지순"
3,상식적으로 상식을 배우는 법,제바스티안 클루스만,2022-04-14,248 쪽,"14,800원",이지윤
4,엑셀로 부자되기,김태형,2022-04-12,312 쪽,"18,000원",-
...,...,...,...,...,...,...
195,"풀스택 서버리스 : 리액트, AWS, 그래프QL을 이용한 최신 애플리케이션 개발",네이더 다빗,2021-07-05,216 쪽,"21,000원",김범준
196,한 권으로 배우는 작고 예쁜 꽃자수,장미나(미나랜드),2021-07-05,240 쪽,"18,000원",-
197,"IT CookBook, 처음 만나는 회로이론(2판)",방성완,2021-07-02,400 쪽,"23,000원",-
198,안전필수 시스템 제어 설계,"김국헌 , 김태효 , 이성섭",2021-07-02,488 쪽,"38,000원",-


#### 6.저장하기

In [66]:
df.to_csv('한빛출판 새로나온 책 목록.csv',index=False)